Here I try the code

In [57]:
import scrapy
import requests
import pandas as pd
from bs4 import BeautifulSoup
from scrapy import Selector
from scrapy.crawler import CrawlerProcess

In [ ]:
ico = ["27201121","27201121"]
start_urls = f"https://or.justice.cz/ias/ui/rejstrik-$firma?ico={ico}&jenPlatne=PLATNE&polozek=1&typHledani=STARTS_WITH"

html = requests.get(url).content

html

In [82]:
#get_url(27082440)
icos = ["27201121","27082440"]

def get_url(ico):
    url_0 = f"https://or.justice.cz/ias/ui/rejstrik-$firma?ico={ico}&jenPlatne=PLATNE&polozek=1&typHledani=STARTS_WITH"
    html_subjektID = requests.get(url_0).content
    sel_id = Selector(text=html_subjektID)
    vypis = sel_id.xpath("//li/a[contains(text(),'Výpis platných')]/@href").extract()
    subjektID = vypis[0].split('subjektId=')[1].split('&')[0]
    url = f'https://or.justice.cz/ias/ui/rejstrik-firma.vysledky?subjektId={subjektID}&typ=PLATNY'
    return url

urls = []

def get_urls(icos):
    for i in icos:
        urls.append(get_url(i))

def get_soup(ico):
    url = get_url(ico)
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'lxml')
    return soup




In [ ]:
ico = "27082440"
url = f"https://or.justice.cz/ias/ui/rejstrik-$firma?ico={ico}&jenPlatne=PLATNE&polozek=1&typHledani=STARTS_WITH"
url2 = "https://or.justice.cz/ias/ui/rejstrik-firma.vysledky?subjektId=701502&typ=PLATNY"
html = requests.get(url2).content
soup = BeautifulSoup(html, 'lxml')

match = soup.find(text="Obchodní firma: ").findNext('span')
match2 = match.findNext('span')


match3 = soup.find(text="Identifikační číslo: ").findNext('span').text

match4 = soup.findAll(class_="nounderline")
match4[5].findNext('span')
print(soup.prettify())



getting the main data points in nice format

In [91]:
def getBasicInfo(icos):
    df = pd.DataFrame(index=icos, columns = ['Obchodní firma', 'Datum vzniku', 'Spisová značka', 'Sídlo', 'Právní forma'])
    for ico in icos:
        soup = get_soup(ico)

        date = soup.find(text="Datum vzniku a zápisu:").findNext('div').findNext('div').text
        df.loc[ico,'Datum vzniku'] = date

        spis = soup.find(text="Spisová značka: ").findNext('span').text
        df.loc[ico,'Spisová značka'] = spis

        obch_firma = soup.find(text="Obchodní firma: ").findNext('span').text
        df.loc[ico,'Obchodní firma'] = obch_firma

        sidlo = soup.find(text="Sídlo: ").findNext('span').findNext('span').text
        df.loc[ico,'Sídlo'] = sidlo

        pravni_forma = soup.find(text="Právní forma: ").findNext('span').text
        df.loc[ico,'Právní forma'] = pravni_forma
    return df

getBasicInfo(icos)


,Obchodní firma,Datum vzniku,Spisová značka,Sídlo,Právní forma
27201121,"APELEN Consulting, spol. s r.o.",5. ledna 2005,C 104017 vedená u Městského soudu v Praze,"Nad Okrouhlíkem 2372/14, Libeň, 182 00 Praha 8",Společnost s ručením omezeným
27082440,Alza.cz a.s.,26. srpna 2003,B 8573 vedená u Městského soudu v Praze,"Jankovcova 1522/53, Holešovice, 170 00 Praha 7",Akciová společnost


filtering the objects so they only include non-empty strings

In [44]:
filter_list=[]

for article in soup.findAll(class_="nounderline"):
    article_text = article.text
    if not article_text:
        pass
    else: 
        filter_list.append(article)

filter_list


[<span class="nounderline">Spisová značka: </span>,
 <span class="nounderline">Obchodní firma: </span>,
 <span class="nounderline">Sídlo: </span>,
 <span class="nounderline">Identifikační číslo: </span>,
 <span class="nounderline">Právní forma: </span>,
 <span class="nounderline">Předmět podnikání: </span>,
 <span class="nounderline">Statutární orgán - představenstvo: </span>,
 <span class="nounderline">Předseda představenstva: </span>,
 <span class="nounderline">Místopředseda představenstva: </span>,
 <span class="nounderline">Místopředseda představenstva: </span>,
 <span class="nounderline">Počet členů: </span>,
 <span class="nounderline">Způsob jednání: </span>,
 <span class="nounderline">Dozorčí rada: </span>,
 <span class="nounderline">člen dozorčí rady: </span>,
 <span class="nounderline">člen dozorčí rady: </span>,
 <span class="nounderline">člen dozorčí rady: </span>,
 <span class="nounderline">Jediný akcionář: </span>,
 <span class="nounderline">Akcie: </span>,
 <span class="n

In [43]:
prvni = filter_list[16].findNext('span')
prvni_text = prvni.text
druhy = prvni.findNext('span')
druhy_text = druhy.text
treti = druhy.findNext('span')
treti_text = treti.text
prvni_text

''

In [ ]:
info = []

def info_attacher(list):
    for object in list:
        while True:
            if object.text == (object+1).text:
                pass
            elif not i.text:
                pass
            else:
                info.append(i.text)

info_attacher(filter_list)
            

In [ ]:
ico = "27201121"
url = f"https://or.justice.cz/ias/ui/rejstrik-$firma?ico={ico}&jenPlatne=PLATNE&polozek=1&typHledani=STARTS_WITH"
   
html = requests.get(url).content
sel = Selector(text=html)
vypis = sel.xpath("//li/a[contains(text(),'Výpis platných')]/@href").extract()

print(vypis)
